In [1]:
import pandas as pd
import json

with open('e7/my_units.txt', 'r') as p: my_units = [unit.rstrip() for unit in p.readlines()]
with open('e7/base_stats.json') as p: base = json.loads(p.read())
data = pd.read_csv("e7/TheHolyGrail.csv")
data = data.rename(columns = {'Unnamed: 0':'name','Unnamed: 1':'skill'})
data[['name']] = data[['name']].fillna(method='ffill') # Fill name into blanks
data = data.fillna(0) # Fill empty

for i, row in data.iterrows():
    name = row['name']
    n = row['skill'][-1]
    if name in base.keys() and n.isdigit():
        # Check for creation of skills dict
        if 'skills' not in base[name].keys(): base[name]['skills'] = {'1':{},'2':{},'3':{}}
        
        base[name]['skills'][n]['targets'] = row['targets']
        
        # Check for stacking skills, then set mult
        if '/' not in str(row['att_rate']): base[name]['skills'][n]['mult'] = float(row['att_rate'])
        else: base[name]['skills'][n]['mult'] = float(row['att_rate'].split('/')[0])
        
        # Check for special scaling, ignoring scaling based off of missing HP
        if row['secondary_stat'] != 0 and 'missing' not in row['secondary_stat']:
            base[name]['skills'][n]['scaling'] = row['secondary_stat']
            base[name]['skills'][n]['scaling_mult'] = row['secondary_scaling%']

In [6]:
# Assumptions
gear = {'atk_flat': 300, 'atk': 2, 'crit': 0.5, 'cd': 0.3, 'spd_flat': 40, 'spd':1.25, 'hp_flat': 1200, 'hp':2, 'def': 100}
target_hp = 10000
target_def = 800

# Calculate Damage
results = {}
for name in base:
    for n in base[name]['skills'].keys():
        if base[name]['skills'][n]['targets'] <= 4: # <= for ST, == for AoE
            # Basic damage calc
            damage = (base[name]['Awake']['Attack']*gear['atk']+gear['atk_flat'])*base[name]['skills'][n]['mult']
            # Special scaling
            if 'scaling' in base[name]['skills'][n].keys():
                if name in ['Ken','Cecilia','Zeno','Dark Corvus'] and 'hp' in base[name]['skills'][n]['scaling'] and 'target' not in base[name]['skills'][n]['scaling']:
                    damage = (base[name]['Awake']['Attack']+gear['atk_flat'])*base[name]['skills'][n]['mult']
                    damage += (base[name]['Awake']['Health']*gear['hp']+gear['hp_flat'])*base[name]['skills'][n]['scaling_mult']
                elif 'hp' in base[name]['skills'][n]['scaling'] and 'target' not in base[name]['skills'][n]['scaling']: damage += (base[name]['Awake']['Health']+gear['hp_flat'])*base[name]['skills'][n]['scaling_mult']
                elif base[name]['skills'][n]['scaling'] == 'def': damage += (base[name]['Awake']['Defense']+gear['def'])*base[name]['skills'][n]['scaling_mult']
                elif 'speed' in base[name]['skills'][n]['scaling']:
                    spd = (base[name]['Awake']['Speed']*gear['spd'])+gear['spd_flat']
                    if name == 'Assassin Cidd':
                        damage *= 1+spd*1.5*base[name]['skills'][n]['scaling_mult']
                        if n == '3': damage += 180*0.003
                    else: damage *= 1+spd*base[name]['skills'][n]['scaling_mult']
                    #if name == 'Vildred': print(name,spd,'spd')
                elif base[name]['skills'][n]['scaling'] == 'target max hp': damage += target_hp*base[name]['skills'][n]['scaling_mult']
                else: print(name,n,base[name]['skills'][n]['scaling'])
                spec_scaling = base[name]['skills'][n]['scaling'].title().replace('Hp','HP')
                if name == 'Zeno': spec_scaling = 'Max HP'
            else: spec_scaling = 'None'

            # Gunther cannot crit, everyone else can
            if 'gunther' not in name.lower(): damage *= 1+(base[name]['Awake']['Critical Hit Chance']+gear['crit'])*(base[name]['Awake']['Critical Hit Damage']+gear['cd']-1)

            # Reduce damage by DEF unless def ignore
            reduced = round(damage/(1+target_def/300),1)
            if n == '3' and name.lower() == 'ludwig':
                reduced = round(damage/(1+(target_def*0.5)/300),1)
                spec_scaling = 'Ignore Def'
            if n == '3' and name.lower() == 'watcher schuri':
                reduced = round(damage,1)
                spec_scaling = 'Ignore Def'
            if n == '2' and name.lower() == 'kise':
                reduced = round(damage/(1+(target_def*0.4)/300),1)
                spec_scaling = 'Ignore Def'

            results[len(results)+1] = {'Name':name, 'Damage':reduced, 'Skill':n, 'Mult': round(base[name]['skills'][n]['mult'],2), 'Scaling':spec_scaling}
            base[name]['skills'][n]['damage'] = reduced
with open('e7/full.json', 'w') as fp: json.dump(base, fp)

In [7]:
sort = 'Damage'
display = pd.DataFrame.from_dict(results, orient='index').sort_values(by=sort,ascending=False).reset_index(drop=True)
display.index = display.index + 1
display[display['Damage'] > 0]

,Name,Damage,Skill,Mult,Scaling
1,Kise,5444.0,3,2.99,Current HP
2,Cidd,5343.8,3,2.99,Speed
3,Watcher Schuri,5070.8,3,1.42,Ignore Def
4,Ken,4436.0,3,2.13,Max HP
5,Yufine,4360.4,3,3.55,None
6,Ludwig,4197.9,2,3.24,None
7,Serila,4010.7,3,3.54,None
8,Guider Aither,3836.4,3,3.20,None
9,Sez,3770.7,3,3.20,None
10,Iseria,3767.5,3,3.37,None


In [4]:
display[display['Name'] == 'Chaos Sect Axe']

,Name,Damage,Skill,Mult,Scaling
31,Chaos Sect Axe,1573.7,2,1.42,None
